This notebook is part of the tutorial on large scale IE under privacy constraints.

Read files from directory. This is expected to be an html file representing email.

In [1]:
def read_file(file_path): 
    with open(file_path, 'r') as f: 
        return (f.read()) 

In [2]:
def load_html_files(dirName):
    file_html = []
    os.chdir(dirName) 
    # iterate through all files 
    for file in os.listdir(): 
        # Check whether file is in text format or not
        file_name = file
        if file.endswith(".html"): 
            file_html.append({"name":file, "text": read_file(file_name)})
    return file_html

Clean the email from HTML format.

In [3]:
def clean_html(file_html):
    file_text=[]
    for fileTextIns in file_html:
        htmlparser = etree.HTMLParser()
        tree = etree.parse(fileTextIns["name"], htmlparser)
        soup = BeautifulSoup(fileTextIns["text"], "html.parser")
        if soup.body:
            for child in soup.body.children:
                if child.name == 'script':
                    child.decompose() 
            file_text.append({"name":fileTextIns["name"], "text": soup.body.get_text('\n'), "tree": tree})
    return file_text

In [4]:
def extract_dates(text):
    months ="January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|Jun|Jul|Aug|Sep|Oct|Nov|Dec"
    years = "([\d]{4}|'[\d]{2}|[\d]{2})"
    days = "Sun|Mon|Tue|Wed|Thu|Fri|Sat|Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday"
    pattern1 = r"(([\d]{1,2})/([\d]{1,2})/("+years+"))"
    pattern2 = r"(([\d]{1,2}\s("+ months +")[,.-]?\s"+years+"))"
    pattern3 = r"((("+ months +")\s[\d]{1,2}[,.-]?\s"+years+"))"
    pattern4 = r"(((0?[1-9]|[12][0-9]|3[01])/(0?[1-9]|1[0-2])/(\d\d))|[012][0-9][0-9][0-9]-[01][0-9]-[0123]?[0-9](t|T)[012][0-9]:[0-5][0-9](:[0-5][0-9])?(.[0-9]{3})?([+-][012][0-9](:)?[0-5][0-9])?Z?)"
    pattern5 = r"(([\d]{1,2}("+ months +")[,.-]?"+years+"))"
    pattern6 = r"(([\d]{1,2}\s*("+ months +")))"
    return [m for m in re.finditer(pattern1 + "|" + pattern2 + "|" + pattern3 + "|" + pattern4 + "|" + pattern5 + "|" + pattern6, text, re.IGNORECASE)]

Features:
1. Is it a min Distance between checkin keyword and the date. Ex. checkin|check-in|departure|depart|departs|leave|leaves|check\sin|travel
2. Is it a Min Distance between checkout keyword and the date. Ex. checkout|check-out|arrival|arrives|reaches|leaves|check\sout
3. Format of the date and the closest checkin keyword-- are they same or not
4. Format of the date and the closest checkout keyword-- are they same or not
5. Is it a Last but min Distance between checkin keyword and the date. Ex. checkin|check-in|departure|depart|departs|leave|leaves|check\sin|travel
6. Is it a Last but min Distance between checkout keyword and the date. Ex. checkout|check-out|arrival|arrives|reaches|leaves|check\sout
7. Are dates and keywords part of same \<text\> tag
8. Min distance to airline code
9. Min distance to any time
10. Distance to the city names

    

In [5]:
def extract_checkin_keywords(text):
    matchCheckinObj = [m for m in re.finditer( r'checkin|check-in|departing|from|departure|depart|departs|leave|leaves|check\sin|travel\s+date', text,re.IGNORECASE)]
    return matchCheckinObj

In [6]:
def extract_checkout_keywords(text):
    matchCheckoutObj = [m for m in re.finditer( r'checkout|check-out|arrival|arriving|To|arrives|reaches|leaves|check\sout', text,re.IGNORECASE)]
    return matchCheckoutObj

In [7]:
def extract_flightNum(text):
    matchAirlineObj = [m for m in re.finditer( r"[1-9A-Z][A-Z]\s?[\s\-]\s?[0-9]{2,4}", text,re.IGNORECASE)]
    return matchAirlineObj

In [8]:
def extract_times(text):
    matchTimeObj = [m for m in re.finditer( r'[012][0-9]:[0-5][0-9](:[0-5][0-9])?(.[0-9]{3})?([+-][012][0-9](:)?[0-5][0-9])?Z?', text,re.IGNORECASE)]
    return matchTimeObj

In [9]:
def extract_pnr(text):
    matchPnrObj = [m for m in re.finditer( r'[A-Z1-9]{6}', text)]
    return matchPnrObj

In [10]:
def extract_class(text):
    matchPnrObj = [m for m in re.finditer( r'Economy|Business|First Class|Executive|Fare', text,re.IGNORECASE)]
    return matchPnrObj

In [11]:
def extract_airports(text):
    airportNames = r'A Coruna|Abbotsford|Aberdeen|Abilene|Abu Dhabi|Abuja|Acapulco|Aden|Adnan Menderes|Ahmedabad|Ajaccio|Akron|Al Ain|Albany|Albuquerque|Alejo Garcia|Aleppo|Alexandria|Alghero|Almeria|Alpena County|Alta|Altoona Blair County|Amarillo|Amman|Amritsar|Amsterdam|Angelholm|Angkor|Antwerp|Arad|Araguaina|Arcata|Aristarchos of Samos|Asheville|Asmara|Aspen Pitkin County|Asuncion|Aswan|Athens|Atlanta|Atlantic City Municipal|Auckland|Augusta State|Aurillac|Austin|Austin Straubel|Avignon Caumont|Bahrain|Baltimore|Bandar Abbas|Bandaranaike|Bangkok|Bangor|Banja Luka|Banjul|Barbados|Barcelona El Prat|Bari|Barquisimeto|Barra|Barrow|Bartolomeu Lysandro|Basseterre|Baton Rouge Metropolitan|Batsfjord|Batumi|Batumi|Beaver Creek|Beijing Capital|Beijing|Beirut Rafic Hariri|Belem|Belfast|Belgrade Nikola Tesla|Belize City|Bellegarde|Bellingham|Ben Gurion|Benazir Bhutto|Benbecula|Bender Qassim|Bengaluru|Bangalore|Benina|Bergen Flesland|Berlevag|Berlin Brandenburg|Berlin Tegel|Bern|Bert Mooney|Bethel|Beziers Vias|Biarritz Parme|Bilbao|Billings|Billund|Bindlacher Berg|Biratnagar|Birmingham|Birmingham Shuttlesworth|Bishop Airport|Bismarck Municipal|Blackpool|Block Island State|Blosser Municipal|Blue Grass|Boa Vista|Bob Hope|Bocas del Toro Isla Colon|Bodensee|Bodo|Boise|Bol|Bole|Bonriki|Boone County|Bora Bora|Bordeaux Mérignac|Borlange|Bornholm|Boryspil|Boston|Boulder Municipal|Bournemouth|Bozeman|Bradley|Hartford Windsor Locks|Brasilia Intl|Bremen|Brest Bretagne|Brisbane|Bristol|Brive Souillac|Brooks Lodge|Broome|Brownsville|Brunei|Brunswick Golden Isles|Brussels|Budapest Ferenc Liszt|Buenos Aires|Buffalo|Bulawayo|Burlington|Cagliari|Cairns|Cairo|Calgary|Cambridge|Campbeltown|Campo Grande|Cancun|Cape Breton|Cape Town|Cardiff|Carriel Sur|Cartagena|Casablanca|Castaway Island Resort|Castlegar|Cat Bi|Cataratas|Central Wisconsin|Cerro Moreno|Chacalluta|Chadron Municipal|Champaign Urbana|Changchun Longjia|Changi|Charles de Gaulle|Charleston|Charlotte|Charlottesville|Charlottetown|Chattanooga|Chengdu|Chennai|Cheong Ju|Cherry Capital|Chester Hawarden|Chiang Mai|Chicago|Chicago Midway|Chiclayo|Chihuahua|Chileka|Chippewa County|Chitose|Chongqing|Christchurch|Christmas Island|Chubu Centrair|Churchill Falls|Chuuk|Cibao|Cincinnati|City of Derry|Ciudad del Carmen|Ciudad Juarez|Ciudad Obregon|Cleveland Airport|Clinton County|Coffee Point|Coimbatore|Cointrin|Cologne Bonn|Colorado Springs|Columbia Metropolitan|Columbus|Comodoro Rivadavia|Concordia|Congonhas Sao Paulo|Constable Pynt|Constanţa|Constantine|Copenhagen|Copernicus|Cordova|Corfu|Corfu|Cork|Cornelio Procopio|Corpus Christi|Corrado Gex|Cortez Municipal|Corumba|Corvallis Municipal|Cote d Azur|Nice|Cozumel|Cranbrook Canadian Rockies|Crescent City|Cristoforo Colombo|Crooked Island|Crotone|Crown Point|Cruzeiro do Sul|Cube Cove |Culiacan|Curaçao International Airport|Curacao International Airport|Curitiba|Cusco|Cuxhaven|Da Nang|Daegu|Dakar|Dala|Dalian Zhoushuizi|Dallas|Dallas Love Field|Damascus|Daniel Oduber Quiros|Darlington|Darwin|Davao City|Dayton|Daytona Beach|Deadhorse|Decatur|Del Bajio|Delta County|Denpasar|Denver|Des Moines|Detroit|Devils Lake Municipal|Dillingham|Diori Hamani|Dodoma|Doha|Domodedovo|Don Mueang|Donegal|Dubai|Dublin|DuBois|Dubuque|Duluth|Dundee|Dunedin|Durango|Durango La Plata|Dusseldorf|Dutchess County|East Midlands|Easterwood|Edinburgh|Edmonton|Eduardo Gomes|Egal|Eindhoven|El Altet|El Alto|El Arish|El Dorado|El Eden|El Matorral|El Paso|El Pucu|El Rahaba|El Salvador|Eldoret|Eloy Alfaro|Entebbe|Eppley Airfield|Ercan|Erie|Erkilet|Ernesto Cortissoz|Esenboga|Eugene|Euro Airport|Exeter|Exuma|Fair Isle|Fairbanks|Faisalabad|Falconara|Faleolo|Falls|Faro|Fayetteville|Federico Fellini|Fernando de Noronha|Fertilia|Fiumicino|Flagstaff Pulliam|Flamingo|Flores|Forbes Field|Ford|Formosa|Fort de France|Fort McMurray|Fort St  John|Fort Wayne|Foz do Iguaçu|Franca|Francistown|Frankfurt|Fredericton|Frejorgues|Fresno Yosemite|Friedman Memorial|Friedrichshafen|Ft Lauderdale|Fuaamotu|Fujairah|Fuzhou Changle|Gagnoa|Galeão|Galileo Galilei|Gallivare|Galway|Garons|Gdansk|General Santos|George Bush|Gillette Campbell|Gimhae|Gimpo|Gisborne|Glasgow|Glasgow Prestwick|Goa|Gold Coast|Goloson|Gondar|Göteborg Landvetter|Gothenburg City|Graciosa Island|Gran Canaria|Granada Jaen|Grand Bahama|Grand Cayman Island|Grand Forks|Grand Rapids|Great Barrier Reef|Great Falls|Greater Binghamton|Greenville|Guadalajara|Guam|Guanajuato|Guangzhou Baiyun|Guarulhos|Guatemala|Guayaquil|Guaymas|Guilin|Guiyang|Gulfport|Gunnison|Guwahati|Hagfors|Haikou|Halifax|Hamburg|Hamilton|Hana|Hanan|Haneda|Hangelar|Hangzhou Xiaoshan|Hannover|Hanoi|Harare|Harbin Taiping|Harbour |Harrisburg|Hartsfield|Hat Yai|Hattiesburg|Haugesund|Havasupai|Heathrow|Hector Airport|Hefei Luogang|Helena|Helsinki|Henri Coanda|Heraklion|Hercilio Luz|Hermanos Serdan|Hermosillo|Herrera|Hewanorra|Hilo|Hilton Head|Hiroshima|Hobby|Hodeida|Hohhot Baita|Homer|Hong Kong|Hongqiao|Honiara|Honolulu|Hoonah|Horta|Houghton County|Houston|Huahine|Huatulco|Hudiksvall|Humberside|Huntsville|Hurghada|Iași|Ikeja|Ilford|Iloilo|Imperial County|In Amenas|Inagua|Inca Manco Capac|Indianapolis|Indira Gandhi|Delhi|Indore|Innsbruck|Inukjuak|Invercargill|Inverness|Iquique|Iquitos|Irkutsk|Isfahan|Isle of Man|Istanbul|Itami|Ixtapa|Jackson Evers|Jackson Hole|Jacksons|Jacksonville|Jeju|Jinan|Jinnah|João Pessoa|John Paul II|Johnston Atoll|Joinville|Jonesboro Municipal|Joplin|Jorge Wilstermann|Jose Marti|Juan Santamaria|Juan Vicente Gomez|Jujuy|Julius Nyerere|Juneau|Kabul|Kahului|Kailua Kona|Kalamazoo Battle Creek|Kano|Kansai|Kansas City|Kapalua|Karratha Airport|Kauai|Kauai|Kaunas|Kavala|Keflavik|Kelowna|Kent|Ketchikan|Key West|Keystone|Kharkiv|Kiev|Kilimanjaro|King Abdulaziz|King Fahd|King Hussein|King Khalid|King Salmon|Kingston|Knevichi|Knock|Knoxville|Kochi|Kolkata|Koltsovo|Kona|Kota Kinabalu|Kotoka|Krabi|Krakow|Kronoberg|Kuala Lumpur|Kuching|Kuopio|Kuusamo|Kuwait|La Aurora|La Braq|La Chinita|La Mesa|La Palma|La Paz|La Tontouta|Lafayette|Lahore|Lake Havasu City|Lake Minchumina|Lamezia Terme|Lampedusa|Lanai|Lancaster|Langkawi|Lansing|Lanzarote|Laoag|Lappeenranta|Laramie|Laredo|Larnaca|Las Americas|Las Vegas|Latrobe|Laughlin Bullhead|Le Havre Octeville|Learmonth|Lebanon Municipal|Leeds Bradford|Lehigh Valley|Leipzig Halle|Leknes|Lewisburg|Lewiston|Lewistown Municipal|Liberal Municipal|Libreville|Lihue|Lille Lesquin|Lilongwe|Lima|Linate|Lincoln|Lisbon Portela|Little Rock National|Liverpool John Lennon|Logan|London|London Biggin Hill|London City|Gatwick|London Heathrow|London Luton|London Stansted|Londrina|Long Beach|Long Island MacArthur|Loreto|Lorient South Brittany|Los Angeles|Los Cabos|Louisville|Luang Prabang|Lubbock|Lubumbashi|Lugano|Lulea|Lungi|Lusaka|Luxembourg Findel|Luxor|Lviv Danylo Halytskyi|Lynden Pindling|Lyon|Maastricht Aachen|Macapa|Macau|Mactan Cebu|Madeira|Madison|Madras Municipal|Madrid|Mae Fah Luang|Malacca|Malaga|Malakal|Male|Malekoula Island|Malpensa|Malta Luga|Managua|Manapouri|Manas|Manchester|Manchester Boston|Mandalay|Mangalore|Manhattan|Manihiki Island|Manila|Mannheim City|Maputo|Marilia|Mariscal Sucre|Marka|Marseille Provence|Marsh Harbour|Mashhad|Mason City Municipal|Massena|Matamoros|Matecana|Matthew Town|Maurice Bishop|Mazatlan|MBS|McAllen Miller|McCarran|McClellan Palomar|Medellin|Mediterranee|Mehamn|Mehrabad|Melbourne|Memphis|Merced Municipal|Mercer County|Mérida|Mexicali|Mexico City|Miami|MidAmerica St Louis|Middle Caicos|Midland|Miles City Municipal|Milwaukee|Minneapolis|Minot|Minsk|Missoula|Mobile|Moenjodaro|Moi|Molde|Molokai|Monastir|Monte Dourado|Monterrey|Montes Claros|Montevideo|Montreal|Montrose|Moore County|Morelia|Moroni|Moscow|Motu Mute|MPLS|Multan|Mumbai|Mundo Maya|Munich|Munster Osnabruck|Muskegon County|Muskrat Dam|Mwana|Myrtle Beach|Mytilene|Nadi|Nairobi|Nanjing|Nanning|Nantucket|Naples|Narita|Narvik|Nashville|Natrona County|Nausori|Navegantes|Nejrab|Nelson|Neuquén|New Bern|New Orleans|New Plymouth|Kennedy|NYC KENNEDY|New York New York|NEW YORK JFK|New York La Guardia|New York LaGuardia|New York Lga|New York Newark|Newark|Newport News|Newquay Cornwall|Ningbo Lishe|Nome|Norderney|Norfolk|North Caicos|Norway House|Norwich|Nuremberg|O Hare|Oakland|Oakland CA|Odessa|Ogdensburg|Ogoki Post|Oklahoma City|Oneida County|Onslow|Ontario|Oradea|Orange County|Oranjestad|Orio al Serio|Orlando|Orly|Osaka|Oslo |Osmani|Ottawa Macdonald Cartier|Oulu|Outagamie|Owen Roberts|Page Municipal|Pago Pago|Palanga|Palenque|Palermo|Palermo|Palm Beach|Palm Springs|Palma de Mallorca|Palmas|Palmerston North|Palmira|Paphos|Paradise Island|Paramaribo|Paris|Paso de los Libres|Pelotas|Penang|Pensacola|Perth|Perugia|Pescara|Petersburg|Philadelphia|Phnom Penh|Phoenix|Phu Bai|Phuket|Piarco|Pico|Piedmont Triad|Piedras Negras|Pilanesberg|Pinto Martins|Pittsburgh|Piura|Plan de Guadalupe|Playa de Oro|Pleso|Pohnpei|Polokwane|Polonia|Ponciano Arriaga|Ponta Delgada|Ponta Pora|Port Allen|Port Angeles|Port au Prince|Port Columbus|Port Harcourt|Port Hedland|Port Moresby|Port of Spain|Portland International|Portland Or|Portland International Jetport|Porto|Porto Velho|Praia|Prague|Prescott Municipal|Prince George|Pristina|Pristina|Providenciales|Pudong|Pueblo Memorial|Puerto Escondido|Puerto Jimenez|Puerto Maldonado|Puerto Suarez|Puerto Vallarta|Pulkovo|Pullman Moscow|Pune|Punta Cana|Punta Gorda|Purdue University|Pyongyang|Qingdao|Quad City|Quebec City|Queenstown|Queretaro|Quesnel|Quetta|Rajiv Gandhi|Hyderabad Hyd|Raleigh County Memorial|Raleigh Durham|Recife|Reggio Calabria|Regina|Region of Waterloo|Rennes|Reno Tahoe|Reynosa|Rhinelander Oneida|Rhodes|Richmond|Riga|Rio Branco|Rio de Janeiro|Rio Grande|Roatan|Roberts|Roberts Field|Rochester|Rochester NY|Rock Springs|Rodriguez Ballon|Rogue Valley|Roissy|Røros|Rorvik|Rostock Laage|Roswell|Rotterdam|Rovaniemi|Sacramento|Saint John|Saint Pierre|Saipan|Salgado Filho|Salina Municipal|Salisbury Ocean|Salt Lake City|Salvador da Bahia|Samos|San Angelo|San Antonio|San Diego|San Francisco|San Jose|San Juan|San Luis|San Marino|Sanaa|Sangster|Santa Barbara|Santa Fe|Santa Genoveva|Santa Maria|Santiago|Santiago Perez|Santo Domingo|Sanya Phoenix|Sao Luis|Sao Tome|Sarasota|Saskatoon|Savannah Hilton Head|Sawyer|Seatac|Seattle|Seoul|Seychelles|Shah Amanat|Shahjalal|Sharm el Sheikh|Shenzhen|Sheremetyevo|Sheridan County|Shreveport|Simon Bolivar|Singapore|Sioux Falls|Sioux Gateway|Sitka|Skopje|Skovde|Sky Harbor|Smithers|Sochi|Soekarno Hatta|Sofia|Sonoma County|Sorocaba|Southampton|Southern Illinois|Southwest Florida|Spokane|Springfield|Springfield Branson|Srinagar|St Croix|St George Municipal|St John s|St Louis|St Martin|St Petersburg|St Thomas|State College|Stavanger|Steamboat Springs|Stephenville|Stewart|Stockholm|Arlanda|Stockholm Bromma|Stockholm Vasteras|Stockton Metropolitan|Stornoway|Strasbourg Entzheim|Stuttgart|Sucre|Sultan Hasanuddin|Sultan Ismail|Summer Beaver|Sundsvall Harnosand|Surabaya|Suvarnabhumi|Svolvær|Sydney|Syracuse|Taba|Tabriz|Tahiti|Taipei|Taiwan Taoyuan|Taiyuan Wusu|Tallahassee|Tampa|Tampico|Tan Son Nhat|Tancredo Neves|Tapachula|Taranto Grottaglie|Tarbes Lourdes Pyrenees|Tashkent|Tbilisi|Ted Stevens Anchorage|Telluride|Tempelhof|Teniente General Benjamin Matienzo|Terrance B Lettsome|The Eastern Iowa|Theodore Francis Green State|Theodore Roosevelt|Thunder Bay|Tianjin Binhai|Tijuana|Timbuktu|Tinian|Tirana|Tiree|Tiruchirapalli|Tobias Bolanos|Tocumen|Tok|Toledo Express|Toncontin|Lester B Pearson Intl|Toronto|Torreón|Toulon Hyères|Toulouse Blagnac|Tours Val de Loire|Tozeur Nefta|Traian Vuia|Trenton Mercer|Treviso|Tri Cities|Tri State|Tribhuvan|Tripoli|Trivandrum|Trondheim|Tucson|Tucurui|Tulsa|Tunis|Tunxi|Turbat|Turin|Turku|Tuxtla Gutierrez|Tweed|U Tapao|Udon Thani|Ulaanbaatar|Umea City|University Park|Ushuaia|Vaasa|Vadsø|Vail Eagle|Val de Caes|Valdez|Valdosta|Valencia|Valencia|Valley|Valparaiso|Vancouver|Vardo|Venice Marco Polo|Veracruz|Vernal|Verona|Victoria|Vienna|Vieux Port|Vigo Peinador|Vigra|Vilhena|Villahermosa|Vilnius|Vilo Acuna|Viracopos Campinas|Virgin Gorda|Viru Viru|Visalia Municipal|Visby|Vitoria Foronda|Vladivostok|Vnukovo|Voronezh|Waco|Waimea Kohala|Wanganui|Warsaw|Washington Reagan|Washington Dulles|Waterford|Waterloo|Watertown|Wattay|Wellington|Westchester County|Westerland Sylt|Westerly|Westport|Whakatane|White Plains|Wichita Falls|Wichita Mid Continent|Wick|Wilkes Barre Scranton|Wilmington|Windsor|Winnipeg|Woodbourne|Wrangell|Wuhan Tianhe|Xi an Xianyang|Xiamen Gaoqi|Xoxocotlan|Yampa Valley|Yangon|Yantai Laishan|Yaounde|Yap|Yarmouth|Yaser Arafat|Yeager|Yellowstone|Yesilkoy International|Yuma|Zagreb|Zakynthos|Zamboanga|Zaporizhia|Zhengzhou Xinzheng|Zielona Gora|Zvartnots|Brussels Charleroi|Zurich'
    airportCodes = r'LCG|YXX|ABZ|ABI|AUH|ABV|ACA|ADE|ADB|AMD|AJA|CAK|AAN|ALB|ABQ|AGT|ALP|ALY|AHO|LEI|APN|ALF|AOO|AMA|AMM|ATQ|AMS|AGH|REP|ANR|ARW|AUX|ACV|SMI|AVL|ASM|ASE|ASU|ASW|ATH|ATL|AIY|AKL|AUG|AUR|AUS|GRB|AVN|BAH|BWI|BND|CMB|BKK|BGR|BNX|BJL|BGI|BCN|BRI|BRM|BRR|BRW|CAW|SKB|BTR|BJF|BUS|BUS|ZBV|PEK|PEK|BEY|BEL|BFS|BEG|BZE|LIG|BLI|TLV|ISB|BEB|BSA|BLR|BLR|BEN|BGO|BVG|BER|TXL|BRN|BTM|BET|BZR|BIQ|BIO|BIL|BLL|BYU|BIR|BHX|BHM|FNT|BIS|BLK|BID|CNK|LEX|BVB|BUR|BOC|FDH|BOO|BOI|BWK|ADD|TRW|HRO|BOB|BOD|BLE|RNN|KBP|BOS|WBU|BOH|BZN|BDL|BDL|BSB|BRE|BES|BNE|BRS|BVE|RBH|BME|BRO|BWN|BQK|BRU|BUD|EZE|BUF|BUQ|BTV|CAG|CNS|CAI|YYC|CBG|CAL|CGR|CUN|YQY|CPT|CWL|CCP|CTG|CMN|CST|YCG|HPH|IGR|CWA|ANF|ARI|CDR|CMI|CGQ|SIN|CDG|CHS|CLT|CHO|YYG|CHA|CTU|MAA|CJJ|TVC|CEG|CNX|ORD|MDW|CIX|CUU|BLZ|CIU|CTS|CKG|CHC|XCH|NGO|ZUM|TKK|STI|CVG|LDY|CME|CJS|CEN|CLE|PLB|CFA|CJB|GVA|CGN|COS|CAE|CSG|CRD|COC|CGH|CNP|CND|CZL|CPH|WRO|CDV|CFU|CFU|ORK|CKO|CRP|AOT|CEZ|CMG|CVO|NCE|NCE|CZM|YXC|CEC|GOA|CRI|CRV|TAB|CZS|CUW|CUL|CUR|CUR|CWB|CUZ|NDZ|DAD|TAE|DKR|BLE|DLC|DFW|DAL|DAM|LIR|MME|DRW|DVO|DAY|DAB|SCC|DEC|BJX|ESC|DPS|DEN|DSM|DTW|DVL|DLG|NIM|DOD|DOH|DME|DMK|CFN|DXB|DUB|DUJ|DBQ|DLH|DND|DUD|DGO|DRO|DUS|POU|EMA|CLL|EDI|YEG|MAO|HGA|EIN|ALC|LPB|AAC|BOG|AXM|FUE|ELP|FMA|SAH|SAL|EDL|MEC|EBB|OMA|ECN|ERI|ASR|BAQ|ESB|EUG|MLH|EXT|GGT|FIE|FAI|LYP|AOI|APW|INL|FAO|FAY|RMI|FEN|AHO|FCO|FLG|BON|FLW|FOE|IMT|FMA|FDF|YMM|YXJ|FWA|IGU|FRC|FRW|FRA|YFC|MPL|FAT|SUN|FDH|FLL|TBU|FJR|FOC|GGN|GIG|PSA|GEV|GWY|FNI|GDN|GES|IAH|GCC|PUS|GMP|GIS|GLA|PIK|GOI|OOL|LCE|GDQ|GOT|GSE|GRW|LPA|GRX|FPO|GCM|GFK|GRR|HTI|GTF|BGM|GSP|GDL|GUM|BJX|CAN|GRU|GUA|GYE|GYM|KWL|KWE|GPT|GUC|GAU|HFS|HAK|YHZ|HAM|YHM|HNM|IUE|HND|BNJ|HGH|HAJ|HAN|HRE|HRB|QFK|MDT|ATL|HDY|PIB|HAU|HAE|LHR|FAR|HFE|HLN|HEL|OTP|HER|FLN|PBC|HMO|HEX|UVF|ITO|HHH|HIJ|HOU|HOD|HET|HOM|HKG|SHA|HIR|HNL|HNH|HOR|CMX|IAH|HUH|HUX|HUV|HUY|HSV|HRG|IAS|LOS|ILF|ILO|IPL|IAM|IGA|JUL|IND|DEL|DEL|IDR|INN|YPH|IVC|INV|IQQ|IQT|IKT|IFN|IOM|IST|ITM|ZIH|JAN|JAC|POM|JAX|CJU|TNA|KHI|JPA|KRK|JON|JOI|JBR|JLN|CBB|HAV|SJO|SVZ|JUJ|DAR|JNU|KBL|OGG|KOA|AZO|KAN|KIX|MCI|JHM|KTA|LIH|LIH|KUN|KVA|KEF|YLW|MSE|KTN|EYW|QKS|HRK|IEV|JRO|JED|DMM|AQJ|RUH|AKN|KIN|VVO|NOC|TYS|COK|CCU|SVX|KOA|BKI|ACC|KBV|KRK|VXO|KUL|KCH|KUO|KAO|KWI|GUA|LAQ|MAR|SAP|SPC|LAP|NOU|LFT|LHE|HII|LMA|SUF|LMP|LNY|LNS|LGK|LAN|ACE|LAO|LPP|LAR|LRD|LCA|SDQ|LAS|LBE|IFP|LEH|LEA|LEB|LBA|ABE|LEJ|LKN|LWB|LWS|LWT|LBL|LBV|LIH|LIL|LLW|LIM|LIN|LNK|LIS|LIT|LPL|BOS|YXU|BQH|LCY|LGW|LHR|LTN|STN|LDB|LGB|ISP|LTO|LRT|LAX|SJD|SDF|LPQ|LBB|FBM|LUG|LLA|FNA|LUN|LUX|LXR|LWO|NAS|LYS|MST|MCP|MFM|CEB|FNC|MSN|MDJ|MAD|CEI|MKZ|AGP|MAK|MLE|LPM|MXP|MLA|MGA|TEU|FRU|MAN|MHT|MDL|IXE|MHK|MHX|MNL|MHG|MPM|MII|UIO|ADJ|MRS|MHH|MHD|MCW|MSS|MAM|PEI|IGA|GND|MZT|MBS|MFE|LAS|CLD|MDE|MPL|MEH|THR|MEL|MEM|MCE|BLF|MID|MXL|MEX|MIA|BLV|MDS|MAF|MLS|MKE|MSP|MOT|MSQ|MSO|MOB|MJD|MBA|MOL|MKK|MIR|MEU|MTY|MOC|MVD|YUL|MTJ|SOP|MLM|HAH|MOW|BOB|MSP|MUX|BOM|FRS|MUC|FMO|MKG|MSA|LAU|MYR|MJT|NAN|NBO|NKG|NNG|ACK|NAP|NRT|NVK|BNA|CPR|SUV|NVT|ALP|NSN|NQN|EWN|MSY|NPL|JFK|JFK|JFK|JFK|LGA|LGA|LGA|EWR|EWR|PHF|NQY|NGB|OME|NRD|ORF|NCA|YNE|NWI|NUE|ORD|OAK|OAK|ODS|OGS|YOG|OKC|UCA|OAJ|ONT|OMR|SNA|AUA|BGY|MCO|ORY|ITM|OSL|ZYL|YOW|OUL|ATW|GCM|PGA|PPG|PLQ|PQM|PMO|PMO|PBI|PSP|PMI|PMW|PMR|CLO|PFO|PID|PBM|CDG|AOL|PET|PEN|PNS|PER|PEG|PSR|PSG|PHL|PNH|PHX|HUI|HKT|POS|PIX|GSO|PDS|NTY|FOR|PIT|PIU|SLW|ZLO|ZAG|PNI|PTG|MES|SLP|PDL|PMG|PAK|CLM|PAP|CMH|PHC|PHE|POM|POS|PDX|PDX|PWM|OPO|PVH|RAI|PRG|PRC|YXS|PRN|PRN|PLS|PVG|PUB|PXM|PJM|PEM|PSZ|PVR|LED|PUW|PNQ|PUJ|PND|LAF|FNJ|TAO|MLI|YQB|ZQN|QRO|YQZ|UET|HYD|HYD|BKW|RDU|REC|REG|YQR|YKF|RNS|RNO|REX|RHI|RHO|RIC|RIX|RBR|GIG|RIG|RTB|ROB|RDM|RST|ROC|RKS|AQP|MFR|CDG|RRS|RVK|RLG|ROW|RTM|RVN|SMF|YSJ|FSP|SPN|POA|SLN|SBY|SLC|SSA|SMI|SJT|SAT|SAN|SFO|SJC|SJU|SBP|SAI|SAH|MBJ|SBA|SAF|GYN|SMA|SCL|AUC|SDQ|SYX|SLZ|TMS|SRQ|YXE|SAV|MQT|SEA|SEA|ICN|SEZ|CGP|DAC|SSH|SZX|SVO|SHR|SHV|SMR|SIN|FSD|SUX|SIT|SKP|KVB|PHX|YYD|AER|CGK|SOF|STS|SOD|SOU|MDH|RSW|GEG|SPI|SGF|SXR|STX|SGU|YYT|STL|SXM|PIE|STT|SCE|SVG|SBS|YJT|SWF|ARN|ARN|BMA|VST|SCK|SYY|SXB|STR|SRE|UPG|JHB|SUR|SDL|SUB|BKK|SVJ|SYD|SYR|TCP|TBZ|PPT|TPE|TPE|TYN|TLH|TPA|TAM|SGN|CNF|TAP|TAR|LDE|TAS|TBS|ANC|TEX|THF|TUC|EIS|CID|PVD|DIK|YQT|TSN|TIJ|TOM|TIQ|TIA|TRE|TRZ|SYQ|PTY|TKJ|TOL|TGU|YYZ|YYZ|TRC|TLN|TLS|TUF|TOE|TSR|TTN|TSF|PSC|HTS|KTM|TIP|TRV|TRD|TUS|TUR|TUL|TUN|TXN|TUK|TRN|TKU|TGZ|HVN|UTP|UTH|ULN|UME|SCE|USH|VAA|VDS|QBF|BEL|VDZ|VLD|VLN|VLC|HRL|VPS|YVR|VAW|VCE|VER|VEL|VRN|YYJ|VIE|QYW|VGO|AES|BVH|VSA|VNO|CYO|VCP|VIJ|VVI|VIS|VBY|VIT|VVO|VKO|VOZ|ACT|MUE|WAG|WAW|DCA|IAD|WAT|ALO|ART|VTE|WLG|HPN|GWT|WST|WSZ|WHK|HPN|SPS|ICT|WIC|AVP|ILM|YQG|YWG|BHE|WRG|WUH|XIY|XMN|OAX|HDN|RGN|YNT|YAO|YAP|YQI|GZA|CRW|WYS|IST|YUM|ZAG|ZTH|ZAM|OZH|CGO|IEG|EVN|CRL|ZRH'
    matchAirportObj = [m for m in re.finditer( airportNames + "|" + airportCodes, text,re.IGNORECASE)]
    return matchAirportObj

In [12]:
def extract_carriers(text):
    carriers = r'SkyWest Airlines|AeroMexico Connect|American Airlines|Air Berlin|Air Canada|Air France|Air India|AirAsia|Aeromexico|Aerolineas Argentinas|Alaska Airlines|AVIANCA|Finnair|Alitalia|JetBlue Airways|British Airways|EVA Air|Air China|China Airlines|Copa Airlines|Cathay Pacific|China Southern Airlines|Delta Air Lines|Norwegian Air Shuttle|Aer Lingus|Emirates|Etihad Airways|Frontier Airlines|Ryanair|Gol|Garuda Indonesia|Allegiant Air|Hawaiian Airlines|Hainan Airlines|TAM Linhas Aereas|JAL|Korean Air|KLM|Kuwait Airways|Air Costa|Lufthansa|SWISS|El Al|Middle East Airlines|Xiamen Airlines|Malaysia Airlines|Air Mauritius|Envoy Air|EgyptAir|China Eastern Airlines|Mokulele Airlines|ANA|Spirit Airlines|Air Japan|Air New Zealand|CSA|Asiana Airlines|Pakistan International Airlines|Philippine Airlines|Qantas|Horizon Air|TAROM|SAS|Brussels Airlines|Singapore Airlines|Aeroflot|Saudia|Sun Country Airlines|Thai Airways International|Turkish Airlines|TAP Portugal|Air Transat|easyJet|United Airlines|SriLankan Airlines|US Airways|Virgin Australia|Vietnam Airlines|Virgin Atlantic|Virgin America|Sunwing|Southwest Airlines|Island Air|Jet Airways|IndiGo|Wizz Air|WestJet'
    matchCarrierObj = [m for m in re.finditer( carriers, text,re.IGNORECASE)]
    return matchCarrierObj

In [13]:
def get_distance(x1, y1, x2, y2):
    return min(abs(x1-x2),  abs(y1-y2))

In [14]:
def get_min_distance(date, LandmarkArr):
    min_dist = sys.maxsize
    if LandmarkArr:
        for lm in LandmarkArr:
            d = get_distance(lm.start(), lm.end(), date.start(), date.end())
            if d < min_dist:
                min_dist = d
                
    if min_dist == sys.maxsize:
        return -1
    else:
        return min_dist

In [15]:
def extract_dates_compute_features(indx, file_text):
    text = file_text[indx]["text"]
    CandidateDates = extract_dates(text)
    
    features = []
    CheckinKeywords = extract_checkin_keywords(text)
    CheckoutKeywords = extract_checkout_keywords(text)
    CandidateFlightNums = extract_flightNum(text)
    CandidateTimes = extract_times(text)
    matchPnrObj = extract_pnr(text)
    ClassKeywords = extract_class(text)
    CandidateAirportCities = extract_airports(text)
    CandidateAirlines = extract_carriers(text)
    
    if CandidateDates:
        for Date in CandidateDates:
            feature = {"name": file_text[indx]["name"], "date": Date.group()}
            feature["checkin_dist"] = get_min_distance(Date, CheckinKeywords)
            feature["checkout_dist"] = get_min_distance(Date, CheckoutKeywords)
            feature["airline_dist"] = get_min_distance(Date, CandidateFlightNums)
            feature["time_dist"] = get_min_distance(Date, CandidateTimes)
            feature["pnr_dist"] = get_min_distance(Date, matchPnrObj)
            feature["class_dist"] = get_min_distance(Date, ClassKeywords)
            feature["airport_dist"] = get_min_distance(Date, CandidateAirportCities)
            feature["carrier_dist"] = get_min_distance(Date, CandidateAirlines)
            feature["dist_start"] = Date.start() / len(text)
            feature["dist_end"] = (len(text) - Date.end())/len(text)
            node= file_text[indx]["tree"].xpath('//*[contains(text(), "'+ Date.group() +'")]')
            if node:
                feature["xpath"] = file_text[indx]["tree"].getpath(node[0])
            else:
                feature["xpath"] = "null"
            features.append(feature)
    return features

In [26]:
def get_labels(gt_filename, features_df):
    gt_data = pd.read_csv(gt_filename, sep='\t', names=['filename','checkinDate','template'])
    labels = []
    for dataindx, data in features_df.iterrows():
        for gtindx, gt in gt_data.iterrows():
            if data["name"] == gt["filename"]:
                if data["date"] == gt["checkinDate"]:
                    labels.append(1)
                else:
                    labels.append(0)
    return labels
    

In [ ]:
def boosting_pr_eyesoff():
    #build_xpath_lookup_table_for_each_field
    #run the weak labelers on all eyes-off dataset
    #iterate through all xpaths for a field for a template on eyes-off data. 
    #Choose the majority answer as the output. 
    #If there is no majority, then output null
    #Output: For every field, for every template, an xpath (possibly null)

In [ ]:
def run_lookup_model():
    #use the table generated above for predictions
    #for each sample in test set, use the above table for predicting the field
    #lookup value in xpath in the sample is the predicition (possibly null)
    #run eval after prediction and generate stats

In [21]:
def eval(model, X_test, Y_test):
    predictions = model.predict(X_test)
    tp = 0
    fp = 0
    fn = 0
    ind=0
    for val in predictions:
        if Y_test[ind] == 1:
            if val == 1:
                tp = tp + 1;
            else:
                fn = fn + 1;
        else:
            if val == 1:
                fp = fp + 1;
        ind = ind + 1
    print ("precision:", tp/ (tp + fp), ", recall:", tp/ (tp+ fn))

In [30]:
def load_data_df(dirName):
    #load flight confirmation emails
    file_html = load_html_files(dirName)

    #clean html, get text, dom tree
    file_text = clean_html(file_html)

    #find all dates in each file and compute features for all dates
    AllFilesfeatures = [];
    for indx in range(len(file_text)):
        AllFilesfeatures = AllFilesfeatures +  extract_dates_compute_features(indx, file_text)

    return pd.DataFrame(AllFilesfeatures, columns=["name", "date", "checkin_dist", "checkout_dist", "airline_dist", "time_dist", "pnr_dist", "dist_start", "dist_end", "class_dist", "airport_dist", "carrier_dist", "xpath"])


In [32]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import re
import math
import datetime
from datetime import date
import sys
from bs4 import BeautifulSoup
from lxml import etree
from html.parser import HTMLParser
import os

gtFileName = r"C:\work\gt.full.tsv"
dirName = r"C:\Users\rakondap\Documents\FlightSamples"
eyesOffdirName = r"C:\Users\rakondap\Documents\FlightSamples\Eyes-off"

#load flight emails features to dataframe and get labels
features_df = load_data_df(dirName)
labels = get_labels(gtFileName, features_df)

#prepare the data for training/testing models: split the data into test and train if needed
trimmed_featuers_df = features_df.drop(["name","date", "xpath"], axis=1)
#X_train, X_test, Y_train, Y_test = train_test_split(trimmed_featuers_df, labels , test_size=0.3)


#load eyes-off data for testing
features_eyesOff_df = load_data_df(eyesOffdirName)
labels_eyesOff = get_labels(gtFileName, features_eyesOff_df)
trimmed_featuers_eyesOff_df = features_eyesOff_df.drop(["name","date", "xpath"], axis=1)

#train LR and evaluate
lr = LogisticRegression(random_state=0, max_iter=1000).fit(trimmed_featuers_df, labels)
print("logistic regression results")
eval(lr, trimmed_featuers_eyesOff_df, labels_eyesOff)

#train RF and evaluate
rf = RandomForestClassifier(max_depth=2, random_state=0).fit(trimmed_featuers_df, labels)
print("Random forest results")
eval(rf, trimmed_featuers_eyesOff_df, labels_eyesOff)




logistic regression results
precision: 0.703125 , recall: 0.625
Random forest results
precision: 0.84375 , recall: 0.75
